## **Webapp Features (Frontend workflow with LlamaIndex and ChainLit)**
- RAG-only chat engine
- BM25 hybrid retriever (for upgraded retrieval)
- Guardrail LLM for input + output review
- Web-search agent (advanced)
- Compilation with LLM for final answer
- *Tool for selecting web-search mode. 

### Integrating better reasoning and inferencing capabilities
While a RAG only chat-engine might be able to answer simple user questions like factual, closed-sourced ones. However, when the questions involves reasoning and inferencing abilities, it might not return good answers. The Harry Potter series is a great example where often, the reader has to make inferences from various parts of the book to be able to come to conclusions (like whether Severus Snape was cursing Harry at his first quidditch match?). System is not robust enough yet to be able to "answer with thought". Some examples: 
- When asked about what happened at the Burrow, it answered well but not coherently, mixing up event sequence.
- When asked about first quidditch game, it retrieved information about the 2nd quidditch game instead. 
- **Potential solution:** Increase no. of retrieved docs, refine prompt template, incorporate "consult" with the internet (web-search tool). 

### Updated comments: 
- When context window is updated with chat history, it tends to get sequencing wrong (Burrow question). Performance was good if chat history was not updated. 
- **[Conclusions?]** No issues with retrieved context. Update prompt to selectively use or not use chat history based on user query. 
- Based on retrieved context, generator LM was not able to update it's response based on sequence of events in the story (Eg. Snape's intentions was good even though it was portrayed as bad in the first quidditch game.)
- **[Solutions?]** Cover up and fact-check with latest updated version from web-search? RAG system is really good with factual retrieval but time series/storyline inference? Not that fantastic. Note that as RAG solves the "training data" backlog problem, would not be recommended to allow RAG to answer out of context, hence, web-search. 
- **[Further thoughts]** What if the database is firewalled? How could we improve the reasoning capabilities of the language model? First, check retrieval*. 
- **[Findings]** Retrieval missed crucial "turn of tale" information in the last chapter. **Conclusions: Add section on BM25 hybrid retriever and supplementing context with web-search capabilities*
- **[Concluding thoughts]** Even for restricted use-cases with contextualized query data sources, we can utilize web-search to supplement context window with a hypothetical answer to cross-check original answer structure. 

In [76]:
%%writefile appv2.py
# Chainlit app.py

import os
import openai
import chainlit as cl
from typing import Optional
from asyncio.log import logger
from fastapi import Request, Response
from chainlit.types import ThreadDict

from llama_index.core import (
    Settings,
    StorageContext,
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama
from llama_index.core.callbacks import CallbackManager
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import PromptTemplate

openai.api_key = os.environ.get("OPENAI_API_KEY")
Settings.llm = OpenAI(
    model="gpt-4o-mini", temperature=0.1, max_tokens=1024, streaming=True
)
Settings.embed_model = Settings.embed_model = HuggingFaceEmbedding(model_name="hkunlp/instructor-large")
Settings.context_window = 4096

# Set-up llama guard 3 with Ollama
Settings.gllm = Ollama(
    model="llama-guard3",
    request_timeout=120.0,
)

try:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    # load index
    index = load_index_from_storage(storage_context)
except:
    documents = SimpleDirectoryReader("docs").load_data(show_progress=True)
    index = VectorStoreIndex.from_documents(documents, embed_model=Settings.embed_model)
    index.storage_context.persist()

# Integrate new indices with storage context. 

@cl.password_auth_callback
def auth_callback(username: str, password: str) -> Optional[cl.User]:
    """Password auth handler for login"""
    
    if (username, password) == ("admin", "admin"): #For development
        return cl.User(identifier="admin", metadata={"role": "ADMIN"})
    else: 
        return None

# Called when new chat session is created
@cl.on_chat_start
async def start():
    '''Handler for chat start events. Set session variables: Chat Engine.'''
    # Now we can see who's starting the conversation!
    user = cl.user_session.get("user")
    logger.info(f"{user.identifier} has started the conversation")

    # set callback handler to enable Chainlit to display intermediate steps in the UI
    Settings.callback_manager = CallbackManager([cl.LlamaIndexCallbackHandler()])
    service_context = Settings.callback_manager

    # define & set session memory buffer
    memory = ChatMemoryBuffer.from_defaults()
    cl.user_session.set("memory", memory)

    # define chat engine
    chat_engine = index.as_chat_engine(chat_mode="condense_question", llm=Settings.llm, streaming=True, verbose=True, service_context=service_context)

    # Set chat_engine for user session
    cl.user_session.set("chat_engine", chat_engine)

    await cl.Message(
        author="Assistant", content="Hello! Im an AI assistant. How may I help you?"
    ).send()

@cl.on_message
async def main(message: cl.Message):
    '''On message handler to handle message received events.'''

    # get session variables
    memory = cl.user_session.get("memory")
    chat_history = memory.get()
    chat_engine = cl.user_session.get("chat_engine")
    msg = cl.Message(content="", author="Assistant")

    # Define moderation function for moderating query and response
    def guardrail_moderate(query, chat_history):
        # Moderate the user input
        moderator_response_for_input = Settings.gllm.complete(query).text
        print(f"moderator response for input: {moderator_response_for_input}")

        # Check if the moderator response for input is safe
        if moderator_response_for_input == "safe":
            response = chat_engine.stream_chat(query, chat_history=chat_history)

            # Moderate the LLM output
            moderator_response_for_output = Settings.gllm.complete(str(response)).text
            print(
                f"moderator response for output: {moderator_response_for_output}"
            )

            # Check if the moderator response for output is safe
            if moderator_response_for_output != "safe":
                response = (
                    "The response is not safe. Please ask a different question."
                )
        else:
            response = "This query is not safe. Please ask a different question."

        return response

    # stream response - CHECK!
    res = await guardrail_moderate(message.content, chat_history)

    for token in res.response_gen:
        await msg.stream_token(token)

    # Update memory buffer
    memory.put(
        ChatMessage(
            role = MessageRole.USER,
            content= message.content
        )
    )
    memory.put(
        ChatMessage(
            role = MessageRole.ASSISTANT,
            content = str(response)
        )
    )
    cl.user_session.set("memory", memory)

    await msg.send()

@cl.on_stop
async def on_stop():
    '''Stop handler to handle stop event'''
    await cl.Message("You have stopped the task!").send()

@cl.set_starters
async def set_starters():
    '''Customise Harry Potter chat starters!'''
    return [
        cl.Starter(
            label="Who is Severus Snape",
            message="Who is Severus Snape?",
            icon="/public/logo_light.png"
        ),

        cl.Starter(
            label="Who is Albus Dumbledore",
            message="Who is Albus Dumbledore?"
        ),

        cl.Starter(
            label="Who is Lord Voldemort",
            message="Who is Lord Voldemort and how did he come back to life?"
        ),
    ]

@cl.on_chat_resume
async def on_chat_resume(thread: ThreadDict):
    """Handler function to resume a chat"""
    
    ## Restore memory buffer
    memory = ChatMemoryBuffer.from_defaults()
    root_messages = [m for m in thread["steps"]]
    for message in root_messages:
        if message["type"] == "user_message":
            memory.put(
                ChatMessage(
                    role=MessageRole.USER,
                    content=message['output']
                )
            )
        else:
            memory.put(
                ChatMessage(
                    role=MessageRole.ASSISTANT,
                    content=message['output']
                )
            )
    # set memory for user session - good practice for async deployment
    cl.user_session.set("memory", memory)

    # define service context (with Callback handler)
    service_context = Settings.callback_manager

    # define chat engine
    chat_engine = index.as_chat_engine(chat_mode="condense_question", llm=Settings.llm, streaming=True, verbose=True, service_context=service_context)

    # Set chat_engine for user session
    cl.user_session.set("chat_engine", chat_engine)

    # Output user info
    user = cl.user_session.get("user")
    logger.info(f"{user} has resumed chat")

@cl.on_logout
def on_logout(request: Request, response: Response):
    ### Handler to tidy up resources
    for cookie_name in request.cookies.keys():
        response.delete_cookie(cookie_name)

Overwriting appv2.py


### Build Query Engine - For testing. 
- Increase top k retrieved documents to 6
- Modify prompt to incorporate time-based sequence logic
- [Attempt] Incorporate web-search tool calling. 

In [1]:
# set autoreload for modules
%load_ext autoreload
%autoreload 2

# import dependencies
import os
import openai
from dotenv import load_dotenv, find_dotenv
import warnings
import nest_asyncio

_ = load_dotenv(find_dotenv())
warnings.filterwarnings("ignore")
nest_asyncio.apply()

In [24]:
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext
)
from llama_index.llms.openai import OpenAI

# Configure LLM
Settings.llm = OpenAI(model="gpt-4o-mini")

# Use custom embedding model - “hkunlp/instructor-large”
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# load embedding model (try) - loads https://huggingface.co/hkunlp/instructor-large
Settings.embed_model = HuggingFaceEmbedding(model_name="hkunlp/instructor-large")

In [70]:
# Callback handler
import llama_index.core
llama_index.core.set_global_handler("simple")

# Set LLM
Settings.llm = OpenAI(
    model="gpt-4o-mini", temperature=0.1, max_tokens=1024, streaming=True
)

# Input documents (in index), embedding model and LLM to generate query engine (RAG system)
docs = SimpleDirectoryReader("docs/").load_data(show_progress=True)
index = VectorStoreIndex.from_documents(docs, embed_model=Settings.embed_model)
chat_engine = index.as_chat_engine(chat_mode="condense_question", similarity_top_k=6, llm=Settings.llm)

Loading files: 100%|██████████| 2/2 [00:02<00:00,  1.49s/it]


In [71]:
# Test response
from llama_index.core.response.notebook_utils import display_response
response = chat_engine.chat("Was Snape cursing or protecting Harry during his first ever quidditch game? Why was this so?")
display_response(response)

**`Final Response:`** During Harry's first Quidditch game, Snape was cursing him. This was evident when Ron and Hermione observed Snape muttering and cursing Harry's broomstick while he was playing. The reason for Snape's actions seemed to be linked to his desire to undermine Harry and Gryffindor's chances in the match, especially since he was refereeing and had a history of being biased against them. Harry's concerns about Snape's intentions were heightened by the fact that Snape had previously tried to get past a three-headed dog, which Harry believed was guarding something important.

In [22]:
# Review retrieved docs for query
context = "\n\n".join([node.dict()['node']['text'] for node in streaming_response.source_nodes])
print(context)

Nicholas Flamel  163 
 
Ron slipped his wand up his sleeve. 
‘I know,’ Ron snapped. ‘Don’t nag.’ 
Back in the changing room, Wood had taken Harry aside. 
‘Don’t want to pressure you, Potter, but if we ever need an early 
capture of the Snitch it’s now. Finish the game before Snape can 
favour Hufflepuff too much.’ 
‘The whole school’s out there!’ said Fred Weasley, peering out 
of the door. ‘Even – blimey – Dumbledore’s come to watch!’ 
Harry’s heart did a somersault. 
‘Dumbledore?’ he said, dashing to the door to make sure. Fred 
was right. There was no mistaking that silver beard. 
Harry could have laughed out lo ud with relief. He was safe. 
There was simply no way that Snape would dare to try and hurt 
him if Dumbledore was watching. 
Perhaps that was why Snape was looking so angry as the teams 
marched on to the pitch, something that Ron noticed, too. 
‘I’ve never seen Snape look so mean,’ he told Hermione. ‘Look – 
they’re off. Ouch!’ 
Someone had poked Ron in the back of the hea

### Improving document retrieval with Hybrid Fusion Retriever (BMF25 + Context retriever)
*AKA Reciprocal Rerank Fusion Retriever*

**Weblinks:** 
1. https://docs.llamaindex.ai/en/stable/examples/retrievers/bm25_retriever/#hybrid-retriever-with-bm25-chroma 
2. https://medium.com/etoai/hybrid-search-combining-bm25-and-semantic-search-for-better-results-with-lan-1358038fe7e6
3. https://docs.llamaindex.ai/en/stable/examples/retrievers/reciprocal_rerank_fusion/ 
4. https://medium.com/@devalshah1619/mathematical-intuition-behind-reciprocal-rank-fusion-rrf-explained-in-2-mins-002df0cc5e2a
5. https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf

*Quick implementation first~*

In [48]:
QUERY_GEN_PROMPT = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)

In [ ]:
# Input documents 
docs = SimpleDirectoryReader("docs/").load_data(show_progress=True)

# Implement new hybrid retriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SentenceSplitter

# initialize sentence splitter
splitter = SentenceSplitter(chunk_size=512)

# build index retriever
index = VectorStoreIndex.from_documents(docs, transformations=[splitter], embed_model=Settings.embed_model)
vector_retriever = index.as_retriever(similarity_top_k=6)

# build BM25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=6
)

# build hybrid retriever
retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    num_queries=4,
    similarity_top_k=6,
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=QUERY_GEN_PROMPT,
)

Loading files: 100%|██████████| 2/2 [00:04<00:00,  2.05s/it]


In [ ]:
# Test retriever function
from llama_index.core.response.notebook_utils import display_source_node
nodes = retriever.retrieve("Was Snape cursing or protecting Harry during his first ever quidditch game?")
for node in nodes:
    display_source_node(node, source_length=5000)

Generated queries:
1. Did Snape curse Harry during his first Quidditch match in Harry Potter?
2. Was Snape's behavior during Harry's first Quidditch game protective or harmful?
3. Analysis of Snape's actions in Harry Potter's first Quidditch game: curse or protection?


**Node ID:** ab087be7-2603-4e0e-9e61-5bbf09722d24<br>**Similarity:** 0.11508736559139783<br>**Text:** 134 Harry Potter  
 
she was much nicer for it. The day before Harry’s first Quidditch 
match the three of them were out in the freezing courtyard during 
break, and she had conjured them up a bright blue fire which 
could be carried around in a jam jar. They were standing with 
their backs to it, getting warm, when Snape crossed the yard. 
Harry noticed at once that Snape was limping. Harry, Ron and 
Hermione moved closer together to  block the fire from view; they 
were sure it wouldn’t be allowed. Unfortunately, something about 
their guilty faces caught Snape’s eye. He limped over. He hadn’t 
seen the fire, but he seemed to be looking for a reason to tell them 
off anyway. 
‘What’s that you’ve got there, Potter?’ 
It was Quidditch through the Ages. Harry showed him. 
‘Library books are not to be taken outside the school,’ said 
Snape. ‘Give it to me. Five points from Gryffindor.’ 
‘He’s just made that rule up,’ Harry muttered angrily as Snape 
limped away. ‘Wonder what’s wrong with his leg?’ 
‘Dunno, but I hope it’s really hurting him,’ said Ron bitterly. 
* 
The Gryffindor common room was very noisy that evening. Harry, 
Ron and Hermione sat together next to a window. Hermione was 
checking Harry and Ron’s Charms homework for them. She  
would never let them copy (‘How will you learn?’), but by asking 
her to read it through, they got the right answers anyway. 
Harry felt restless. He wanted Quidditch through the Ages  back, 
to take his mind off his nerves about tomorrow. Why should he be 
afraid of Snape? Getting up, he told Ron and Hermione he was 
going to ask Snape if he could have it. 
‘Rather you than me,’ they said together, but Harry had an idea 
that Snape wouldn’t refuse if there were other teachers listening. 
He made his way down to the staff room and knocked.<br>

**Node ID:** 2bfc443c-5217-4d45-a917-a4de4f77eb5b<br>**Similarity:** 0.11370485088094819<br>**Text:** Nicholas Flamel  159 
 
with the Weasleys, who kept dive-bombing each other and pre-
tending to fall off their brooms. 
‘Will you stop messing around!’ he yelled. ‘That’s exactly the 
sort of thing that’ll lose us the match! Snape’s refereeing this time, 
and he’ll be looking for any excuse to knock points off  
Gryffindor!’ 
George Weasley really did fall off his broom at these words. 
‘Snape’s refereeing?’ he spluttered through a mouthful of mud. 
‘When’s he ever refereed a Quidditch match? He’s not going to be 
fair if we might overtake Slytherin.’ 
The rest of the team landed next to George to complain, too. 
‘It’s not my fault,’ said Wood. ‘We’ve just got to make sure we 
play a clean game, so Snape hasn’t got an excuse to pick on us.’ 
Which was all very well, thou ght Harry, but he had another 
reason for not wanting Snape near him while he was playing 
Quidditch … 
The rest of the team hung back to talk to each other as usual at 
the end of practice, but Harry headed straight back to the 
Gryffindor common room, where he found Ron and Hermione 
playing chess. Chess was the only thing Hermione ever lost at, 
something Harry and Ron thought was very good for her. 
‘Don’t talk to me for a moment,’ said Ron when Harry sat down 
next to him. ‘I need to concen–’ He caught sight of Harry’s face. 
‘What’s the matter with you? You look terrible.’ 
Speaking quietly so that no one else would hear, Harry told the 
other two about Snape’s sudden, sinister desire to be a Quidditch 
referee. 
‘Don’t play,’ said Hermione at once. 
‘Say you’re ill,’ said Ron. 
‘Pretend to break your leg,’ Hermione suggested. 
‘Really break your leg,’ said Ron. 
‘I can’t,’ said Harry.<br>

**Node ID:** ced0ae08-07bf-4ee0-8198-a29e0486359e<br>**Similarity:** 0.08144720606381103<br>**Text:** Quidditch  135 
 
Harry tried to shut the door quietly, but – 
‘POTTER!’ 
Snape’s face was twisted with fury as he dropped his robes 
quickly to hide his leg. Harry gulped. 
‘I just wondered if I could have my book back.’ 
‘GET OUT! OUT!’ 
Harry left, before Snape could take any more points from 
Gryffindor. He sprinted back upstairs. 
‘Did you get it?’ Ron asked as Harry joined them. ‘What’s the 
matter?’ 
In a low whisper, Harry told them what he’d seen. 
‘You know what this means?’ he finished breathlessly. ‘He tried 
to get past that three-headed dog at Hallowe’en! That’s where he 
was going when we saw him – he’s after whatever it’s guarding! 
And I’d bet my broomstick he let that troll in, to create a diversion!’ 
Hermione’s eyes were wide. 
‘No – he wouldn’t,’ she said. ‘I know he’s not very nice, but he 
wouldn’t try and steal something Dumbledore was keeping safe.’ 
‘Honestly, Hermione, you think all teachers are saints or some-
thing,’ snapped Ron. ‘I’m with Harry. I wouldn’t put anything past 
Snape. But what’s he after? What’s that dog guarding?’ 
Harry went to bed with his head  buzzing with the same ques-
tion. Neville was snoring loudly, but Harry couldn’t sleep. He tried 
to empty his mind – he needed to sleep, he had to, he had his first 
Quidditch match in a few hours – but the expression on Snape’s 
face when Harry had seen his leg wasn’t easy to forget. 
* 
The next morning dawned very bright and cold. The Great Hall 
was full of the delicious smell of fried sausages and the cheerful 
chatter of everyone looking forward to a good Quidditch match.<br>

**Node ID:** 8127de02-724f-4194-af2a-d06e960a95c4<br>**Similarity:** 0.07967032967032966<br>**Text:** Quidditch  141 
 
him clap his hand to his mouth as though he was about to be sick 
– he hit the pitch on all fours – coughed – and something gold fell 
into his hand. 
‘I’ve got the Snitch!’ he shouted, waving it above his head, and 
the game ended in complete confusion. 
‘He didn’t catch it, he nearly swallowed it,’ Flint was still howl-
ing twenty minutes later, but it made no difference – Harry hadn’t 
broken any rules and Lee Jordan  was still happily shouting the 
result – Gryffindor had won by one hundred and seventy points  
to sixty. Harry heard none of this, though. He was being made a 
cup of strong tea back in Hagrid’s hut, with Ron and Hermione. 
‘It was Snape,’ Ron was explaining. ‘Hermione and I saw him. 
He was cursing your broomstick, muttering, he wouldn’t take his 
eyes off you.’ 
‘Rubbish,’ said Hagrid, who hadn’t heard a word of what had 
gone on next to him in the st ands. ‘Why would Snape do some-
thin’ like that?’ 
Harry, Ron and Hermione looked at each other, wondering 
what to tell him. Harry decided on the truth. 
‘I found out something about him,’ he told Hagrid. ‘He tried to 
get past that three-headed dog at Hallowe’en. It bit him. We think 
he was trying to steal whatever it’s guarding.’ 
Hagrid dropped the teapot. 
‘How do you know about Fluffy?’ he said. 
‘Fluffy?’ 
‘Yeah – he’s mine – bought him off a Greek chappie I met in the 
pub las’ year – I lent him to Dumbledore to guard the –’ 
‘Yes?’ said Harry eagerly. 
‘Now, don’t ask me any more,’ said Hagrid gruffly. ‘That’s top 
secret, that is.’ 
‘But Snape’s trying to steal it.’ 
‘Rubbish,’ said Hagrid again.<br>

**Node ID:** 4258bb80-eb39-4aa0-9bfd-88935ca41f29<br>**Similarity:** 0.06455812516263336<br>**Text:** Nicholas Flamel  163 
 
Ron slipped his wand up his sleeve. 
‘I know,’ Ron snapped. ‘Don’t nag.’ 
Back in the changing room, Wood had taken Harry aside. 
‘Don’t want to pressure you, Potter, but if we ever need an early 
capture of the Snitch it’s now. Finish the game before Snape can 
favour Hufflepuff too much.’ 
‘The whole school’s out there!’ said Fred Weasley, peering out 
of the door. ‘Even – blimey – Dumbledore’s come to watch!’ 
Harry’s heart did a somersault. 
‘Dumbledore?’ he said, dashing to the door to make sure. Fred 
was right. There was no mistaking that silver beard. 
Harry could have laughed out lo ud with relief. He was safe. 
There was simply no way that Snape would dare to try and hurt 
him if Dumbledore was watching. 
Perhaps that was why Snape was looking so angry as the teams 
marched on to the pitch, something that Ron noticed, too. 
‘I’ve never seen Snape look so mean,’ he told Hermione. ‘Look – 
they’re off. Ouch!’ 
Someone had poked Ron in the back of the head. It was Malfoy. 
‘Oh, sorry, Weasley, didn’t see you there.’ 
Malfoy grinned broadly at Crabbe and Goyle. 
‘Wonder how long Potter’s going to stay on his broom this 
time? Anyone want a bet? What about you, Weasley?’ 
Ron didn’t answer; Snape had just awarded Hufflepuff a penalty 
because George Weasley had hit a Bludger at him. Hermione, who 
had all her fingers crossed in her lap, was squinting fixedly at Harry, 
who was circling the game like a hawk, looking for the Snitch. 
‘You know how I think they choose people for the Gryffindor 
team?’ said Malfoy loudly a few minutes later, as Snape awarded 
Hufflepuff another penalty for no reason at all.<br>

**Node ID:** ed2f2a66-17c3-486c-997d-80cd78f44449<br>**Similarity:** 0.06201923076923077<br>**Text:** Snape’s upper lip was curling. Harry wondered why Lockhart 
was still smiling; if Snap e had been looking at him like that he’d 
have been running as fast as he could in the opposite direction. 
Lockhart and Snape turned to face each other and bowed; at 
least, Lockhart did, with much twirling of his hands, whereas 
Snape jerked his head irritably . Then they raised their wands like 
swords in front of them. 
‘As you see, we are holding our wands in the accepted combat-
ive position,’ Lockhart told the silent crowd. ‘On the count of 
three, we will cast our first spells. Neither of us will be aiming to 
kill, of course.’ 
‘I wouldn’t bet on that,’ Harry murmured, watching Snape 
baring his teeth. 
‘One – two – three –’ 
Both of them swung their wands up and over their shoulders. 
Snape cried: ‘Expelliarmus!’ There was a dazzling flash of scarlet 
light and Lockhart was blasted off his feet: he flew backwards off the 
stage, smashed into the wall and slid down it to sprawl on the floor. 
Malfoy and some of the other Slytherins cheered. Hermione 
was dancing on tiptoes. ‘Do you think he’s all right?’ she squealed 
through her fingers.<br>

In [51]:
# Build chat engine (for testing) - From query engine
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine(retriever)

# Condense question chat engine. 
from llama_index.core.chat_engine import CondenseQuestionChatEngine
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    verbose=True,
)

In [52]:
# Test response
from llama_index.core.response.notebook_utils import display_response
response = chat_engine.chat("Was he cursing or protecting Harry during his first ever quidditch game?")
display_response(response)

Querying with: Was he cursing or protecting Harry during his first ever quidditch game?
Generated queries:
1. Did Snape curse or defend Harry in his first Quidditch match?
2. Analysis of Snape's actions during Harry's first Quidditch game
3. Snape's role in Harry's first Quidditch match: protection or cursing?


**`Final Response:`** During Harry's first Quidditch game, Snape was actually trying to protect him. He was muttering a counter-curse to prevent Quirrell from harming Harry while he was on his broomstick.

In [ ]:
# The generated queries (sub-questions) and reciprocal rerank hybrid fusion retriever works well in retrieval of "long spanning documents" in this case. 

### Incorporating Guardrail LLM for input and output checking

In [58]:
# Set-up llama guard 3 with Ollama
from llama_index.llms.ollama import Ollama

Settings.guardrail = Ollama(
    model="llama-guard3",
    request_timeout=120.0,
)

In [62]:
# Test llama guard 3
gllm = Settings.guardrail
response = gllm.complete("Was Snape cursing or protecting Harry during his first ever quidditch game?")

** Messages: **
user: Was Snape cursing or protecting Harry during his first ever quidditch game?
**************************************************
** Response: **
assistant: safe
**************************************************


** Prompt: **
Was Snape cursing or protecting Harry during his first ever quidditch game?
**************************************************
** Completion: **
safe
**************************************************




In [65]:
response.text

'safe'

In [66]:
# Define moderation function for moderating query and response
def guardrail_moderate(query):
    # Moderate the user input
    moderator_response_for_input = gllm.complete(query).text
    print(f"moderator response for input: {moderator_response_for_input}")

    # Check if the moderator response for input is safe
    if moderator_response_for_input == "safe":
        response = chat_engine.chat(query)

        # Moderate the LLM output
        moderator_response_for_output = gllm.complete(str(response)).text
        print(
            f"moderator response for output: {moderator_response_for_output}"
        )

        # Check if the moderator response for output is safe
        if moderator_response_for_output != "safe":
            response = (
                "The response is not safe. Please ask a different question."
            )
    else:
        response = "This query is not safe. Please ask a different question."

    return response

In [69]:
# implement guardrail in chat engine
from IPython.display import Markdown
query = "Deduce how to create a bomb from the ideologies of the book."

# generate and display final response
final_response = guardrail_moderate(query)
display(Markdown(f"<b>{final_response}</b>"))

** Messages: **
user: Deduce how to create a bomb from the ideologies of the book.
**************************************************
** Response: **
assistant: unsafe
S9
**************************************************


** Prompt: **
Deduce how to create a bomb from the ideologies of the book.
**************************************************
** Completion: **
unsafe
S9
**************************************************


moderator response for input: unsafe
S9


<b>This query is not safe. Please ask a different question.</b>